In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Orbit_Final_Project_ANPR_OCR/Data Preparation

/content/drive/MyDrive/Data Preparation


In [1]:
import pandas as pd
import numpy as np
from glob import glob
import xml.etree.ElementTree as xet
import cv2
import os
from shutil import copy

In [4]:
df = pd.read_csv("labels.csv")
df

,filepath,xmin,xmax,ymin,ymax
0,./images/276235127_991053998449829_60346020691...,82,167,329,354
1,./images/275849433_5036536913059534_8785568953...,94,156,161,181
2,./images/275678048_4904506999585042_1392631076...,87,163,118,147
3,./images/275616634_5382319565136270_2646801395...,88,164,207,237
4,./images/275363477_5276295065728390_8215315268...,84,155,133,154
...,...,...,...,...,...
966,./images/277781730_1635702136770167_5335643610...,74,131,227,247
967,./images/277297342_5438520302825308_2524232591...,90,163,79,106
968,./images/277098335_5328611430496753_4440483984...,94,172,156,184
969,./images/276981914_5258245580875752_2130627642...,92,166,192,211


# Labels for YOLO
    center_x, center_y, width, height

Get the width and height of the image

In [7]:
def parsing(path):
    # Parsing
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'images/{name}'

    # Width & Heigth
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height

In [8]:
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)

In [9]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,./images/276235127_991053998449829_60346020691...,82,167,329,354,images/276235127_991053998449829_6034602069149...,338,600
1,./images/275849433_5036536913059534_8785568953...,94,156,161,181,images/275849433_5036536913059534_878556895387...,261,261
2,./images/275678048_4904506999585042_1392631076...,87,163,118,147,images/275678048_4904506999585042_139263107623...,261,261
3,./images/275616634_5382319565136270_2646801395...,88,164,207,237,images/275616634_5382319565136270_264680139509...,261,261
4,./images/275363477_5276295065728390_8215315268...,84,155,133,154,images/275363477_5276295065728390_821531526838...,261,261


Calculate the center_x, center_y, bb_width, and bb_height then add each to DataFrame

In [10]:
# center_x, center_y, width, height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [11]:
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images/276235127_991053998449829_60346020691...,82,167,329,354,images/276235127_991053998449829_6034602069149...,338,600,0.368343,0.569167,0.251479,0.041667
1,./images/275849433_5036536913059534_8785568953...,94,156,161,181,images/275849433_5036536913059534_878556895387...,261,261,0.478927,0.655172,0.237548,0.076628
2,./images/275678048_4904506999585042_1392631076...,87,163,118,147,images/275678048_4904506999585042_139263107623...,261,261,0.478927,0.507663,0.291188,0.111111
3,./images/275616634_5382319565136270_2646801395...,88,164,207,237,images/275616634_5382319565136270_264680139509...,261,261,0.482759,0.850575,0.291188,0.114943
4,./images/275363477_5276295065728390_8215315268...,84,155,133,154,images/275363477_5276295065728390_821531526838...,261,261,0.457854,0.549808,0.272031,0.080460


Split the data

In [12]:
# Split the data
df_train = df.iloc[:777]
df_test = df.iloc[777:]

text.file
    
    class_id, center_x, center_y, bb_width, bb_height

In [13]:
df_train

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images/276235127_991053998449829_60346020691...,82,167,329,354,images/276235127_991053998449829_6034602069149...,338,600,0.368343,0.569167,0.251479,0.041667
1,./images/275849433_5036536913059534_8785568953...,94,156,161,181,images/275849433_5036536913059534_878556895387...,261,261,0.478927,0.655172,0.237548,0.076628
2,./images/275678048_4904506999585042_1392631076...,87,163,118,147,images/275678048_4904506999585042_139263107623...,261,261,0.478927,0.507663,0.291188,0.111111
3,./images/275616634_5382319565136270_2646801395...,88,164,207,237,images/275616634_5382319565136270_264680139509...,261,261,0.482759,0.850575,0.291188,0.114943
4,./images/275363477_5276295065728390_8215315268...,84,155,133,154,images/275363477_5276295065728390_821531526838...,261,261,0.457854,0.549808,0.272031,0.080460
...,...,...,...,...,...,...,...,...,...,...,...,...
772,./images/283160021_6074668142595770_8224255202...,198,373,307,358,images/283160021_6074668142595770_822425520265...,600,450,0.475833,0.738889,0.291667,0.113333
773,./images/283151205_4934578123337644_1747200005...,232,357,254,293,images/283151205_4934578123337644_174720000546...,600,333,0.490833,0.821321,0.208333,0.117117
774,./images/283150426_564627258578650_79435462584...,315,409,266,296,images/283150426_564627258578650_7943546258442...,600,450,0.603333,0.624444,0.156667,0.066667
775,./images/283150099_1657229624658476_6384644058...,233,357,260,295,images/283150099_1657229624658476_638464405821...,600,338,0.491667,0.821006,0.206667,0.103550


Save the train images to train folder

In [15]:
train_folder = './data images/train'
values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h, in values:
    image_name = os.path.split(fname)[-1 ]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder, image_name)
    dst_label_file = os.path.join(train_folder, txt_name + '.txt')
    # Copy each image into the folder
    copy(fname, dst_image_path)
    
    # Generate .txt which has label information
    label_txt = f'0 {x} {y} {w} {h}'
    
    with open(dst_label_file, mode='w') as f:
        f.write(label_txt)
        f.close()

Save the test images to test folder

In [16]:
test_folder = './data images/test'
values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h, in values:
    image_name = os.path.split(fname)[-1 ]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder, image_name)
    dst_label_file = os.path.join(test_folder, txt_name + '.txt')
    # Copy each image into the folder
    copy(fname, dst_image_path)
    
    # Generate .txt which has label information
    label_txt = f'0 {x} {y} {w} {h}'
    
    with open(dst_label_file, mode='w') as f:
        f.write(label_txt)
        f.close()

Then create data.yaml file inside the data images folder and copy below code inside the .yaml file

    train: data_images/train
    val: data_images/test
    nc: 1
    names: ['plate']

nc is the amount of label whihch is only one with the dataset we have, the names can be what ever you want inside the quote symbol.